In [ ]:
import pandas
import os
import numpy
import plotly.express
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix, precision_score, recall_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef, average_precision_score
from sklearn.tree import plot_tree
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import xgboost
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
import glob

df = []

csv_files = glob.glob('data/Physical dataset/*.csv')
print(csv_files)
for csv_file in csv_files:
    delimiter = '\t'
    encoding="utf-16le"
    if csv_file.endswith("4.csv"):
        delimiter = ','
        encoding = "utf-8"
    df_temp = pandas.read_csv(csv_file, encoding=encoding, delimiter=delimiter)
    df.append(df_temp)

df  = pandas.concat(df, ignore_index=True)


In [ ]:
import pandas as pd
import numpy as np

# Remplacer les valeurs non nulles dans Label_n par celles de Lable_n
df['Label_n'] = df['Label_n'].combine_first(df['Lable_n'])

# Supprimer la colonne Lable_n
df.drop('Lable_n', axis=1, inplace=True)

# Afficher le DataFrame résultant
nan_ratios = df.isna().mean()   
print(nan_ratios)


In [ ]:
nb_attaque= df['Label'].value_counts()
print(nb_attaque)
ratio = (nb_attaque.iloc[1] + nb_attaque.iloc[2]+ nb_attaque.iloc[3] + nb_attaque.iloc[5])/(nb_attaque.iloc[0]+nb_attaque.iloc[1]+nb_attaque.iloc[2]+ nb_attaque.iloc[3] + nb_attaque.iloc[4] +nb_attaque.iloc[5])
print(ratio*100)

Ici on test sur des partie du dataframe pour voir le nombre minimal de data à prendre pour avoir un bon F1 score

In [ ]:
# Create reduced datasets
train_sizes = [100, 500,1000,5000,10000]

# Initialize lists to store results

recall_scores = []
accuracy_scores = []

clf = DecisionTreeClassifier()


for size in train_sizes:

    fraudulent_entries = df[df['Label_n'] == 1].sample(n=int(size * ratio) +1)
    non_fraudulent_entries = df[df['Label_n'] == 0].sample(n=int(size * (1-ratio)))

    df_reduced = pandas.concat([fraudulent_entries, non_fraudulent_entries])

    Y_red=df_reduced['Label_n'].copy()
    X_red=df_reduced.drop(['Label_n', 'Time','Label'], axis=1)
    X_train, X_test, Y_train, Y_test = train_test_split(X_red, Y_red, test_size=0.2, random_state=42)

    clf = DecisionTreeClassifier()
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
   
    recall_scores.append(recall_score(Y_test, Y_pred))
    accuracy_scores.append(accuracy_score(Y_test, Y_pred))
   

In [ ]:
# Plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(train_sizes, recall_scores, label='Recall')
plt.plot(train_sizes, accuracy_scores, label='Accuracy')

plt.title('Decision Tree for Different Training Set Sizes')
plt.xlabel('Training Set Size')
plt.ylabel('Score')
plt.legend()
plt.show()

In [ ]:
train_sizes = [100, 500,1000,5000,10000]


recall_scores = []
accuracy_scores = []

clf = xgboost.XGBClassifier()


for size in train_sizes:

    fraudulent_entries = df[df['Label_n'] == 1].sample(n=int(size * ratio) +1)
    non_fraudulent_entries = df[df['Label_n'] == 0].sample(n=int(size * (1-ratio)))

    df_reduced = pandas.concat([fraudulent_entries, non_fraudulent_entries])

    Y_red=df_reduced['Label_n'].copy()
    X_red=df_reduced.drop(['Label_n', 'Time','Label'], axis=1)
    X_train, X_test, Y_train, Y_test = train_test_split(X_red, Y_red, test_size=0.2, random_state=42)

    clf = DecisionTreeClassifier()
    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
   
    recall_scores.append(recall_score(Y_test, Y_pred))
    accuracy_scores.append(accuracy_score(Y_test, Y_pred))
   

In [ ]:
# Plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(train_sizes, recall_scores, label='Recall')
plt.plot(train_sizes, accuracy_scores, label='Accuracy')

plt.title('Xgboost for Different Training Set Sizes')
plt.xlabel('Training Set Size')
plt.ylabel('Score')
plt.legend()
plt.show()

In [ ]:
train_sizes = [100, 500,1000,5000,10000]


recall_scores = []
accuracy_scores = []

clf = SVC(kernel='rbf') 



for size in train_sizes:
    fraudulent_entries = df[df['Label_n'] == 1].sample(n=int(size * ratio) + 1)
    non_fraudulent_entries = df[df['Label_n'] == 0].sample(n=int(size * (1 - ratio)))

    df_reduced = pd.concat([fraudulent_entries, non_fraudulent_entries])

    Y_red = df_reduced['Label_n'].copy()
    X_red = df_reduced.drop(['Label_n', 'Time', 'Label'], axis=1)

    X_train, X_test, Y_train, Y_test = train_test_split(X_red, Y_red, test_size=0.2, random_state=42)

    clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)

    recall_scores.append(recall_score(Y_test, Y_pred))
    accuracy_scores.append(accuracy_score(Y_test, Y_pred))
   

In [ ]:
# Plot the results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(train_sizes, recall_scores, label='Recall')
plt.plot(train_sizes, accuracy_scores, label='Accuracy')

plt.title('SVM for Different Training Set Sizes')
plt.xlabel('Training Set Size')
plt.ylabel('Score')
plt.legend()
plt.show()

Ici on entraine sur tous le dataframe

In [ ]:
train_sizes = [100, 500,1000,5000,10000]

recall_scores = []
accuracy_scores = []

knn = KNeighborsClassifier(n_neighbors=1)

for size in train_sizes:

    fraudulent_entries = df[df['Label_n'] == 1].sample(n=int(size * ratio) +1)
    non_fraudulent_entries = df[df['Label_n'] == 0].sample(n=int(size * (1-ratio)))

    df_reduced = pandas.concat([fraudulent_entries, non_fraudulent_entries])

    Y_red=df_reduced['Label_n'].copy()
    X_red=df_reduced.drop(['Label_n', 'Time','Label'], axis=1)
    X_train, X_test, Y_train, Y_test = train_test_split(X_red, Y_red, test_size=0.2, random_state=42)

    knn.fit(X_train, Y_train)
    Y_pred = knn.predict(X_test)
   
    recall_scores.append(recall_score(Y_test, Y_pred))
    accuracy_scores.append(accuracy_score(Y_test, Y_pred))
   

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(train_sizes, recall_scores, label='Recall')
plt.plot(train_sizes, accuracy_scores, label='Accuracy')

plt.title('KNN for Different Training Set Sizes')
plt.xlabel('Training Set Size')
plt.ylabel('Score')
plt.legend()
plt.show()